# Simulación de caracteristicas de máquina de inducción

Ésta aplicación permite la simulación de características de una máquina de inudcción.

In [1]:
%matplotlib widget

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from core.im import im
import ipywidgets
plt.style.use('presentation.mplstyle')

In [3]:

plt.ioff()
plt.clf()

fig, axes = plt.subplots(ncols=2, figsize=(9,4), frameon=False)
fig.canvas.toolbar_visible = False

mi_1 = im()

n_array = np.linspace(0,1499+200,1000)
mi_1.set_motor('abb_22kW')
mi_1.update(n_array)


line_T_u  = axes[0].plot(n_array,mi_1.T_u)
line_P_1 = axes[1].plot(n_array,np.abs(mi_1.P_1))
line_Q_1 = axes[1].plot(n_array,np.abs(mi_1.Q_1))

line_T_u_nom = axes[0].plot(n_array,(n_array-n_array) + mi_1.T_u_nom)
#line_I_1_nom = axes[1].plot(n_array,(n_array-n_array) + mi_1.I_nom)

line_T_u_state = axes[0].plot(mi_1.n_nom, mi_1.T_u_nom,'o')
line_P_1_state = axes[1].plot(mi_1.n_nom, mi_1.P_1[0],'o')
line_Q_1_state = axes[1].plot(mi_1.n_nom, mi_1.Q_1[0],'o')

sld_R2 = ipywidgets.FloatSlider(orientation='horizontal',description = u"R rotor:", 
                                value=0.0, min=0.0,max= mi_1.R_2_nominal*10, 
                                step=mi_1.R_2_nominal)

sld_U1 = ipywidgets.FloatSlider(orientation='horizontal',description = u"U:", 
                                value=mi_1.U_n, min=0.0,max= mi_1.U_n*1.2,
                                step=1)

sld_freq = ipywidgets.FloatSlider(orientation='horizontal',description = u"freq:", 
                                value=mi_1.freq, min=0.0,max= mi_1.freq*1.2,
                                 step = 1)

sld_tau = ipywidgets.FloatSlider(orientation='horizontal',description = u"T_u:", 
                                value=0.2, min=-mi_1.T_u_nom*1.5,max= mi_1.T_u_nom*1.5,
                                 step = 1)

ckbox_escalar = ipywidgets.Checkbox(
    value=False,
    description='CTRL Escalar',
    disabled=False
)


prog_c = ipywidgets.IntProgress(
    value=100,
    min=0,
    max=120,
    step=1,
    description='Carga:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal' 
)

drop_mtype = ipywidgets.Dropdown(
    options=[('ABB 3kW 2pp', 'abb_3kW'),
             ('ABB 22kW 2pp','abb_22kW'), 
             ('ABB 90kW 2pp', 'abb_90kW'),
             ('Rotor bobinado 22 kW 2pp', 'abb_22kWw'),
            ],
    value='abb_22kW',
    description='Motor:',
    disabled=False,
)

btn_reset = ipywidgets.Button(
    description='Reiniciar',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Reiniciar',
    icon=''
)

fig.tight_layout()
axes[0].set_title('Par en función de la velocidad')
axes[1].set_title('Potencias en función de la velocidad (kVA)');

In [4]:
def initial(change):
    
    mtype = drop_mtype.value
    mi_1.set_motor(mtype)
    n_array = np.linspace(0,1499+200,500)
    mi_1.update(n_array)

    mi_1.R_2_start_0 =  mi_1.R_2_start
    mi_1.R_2_max_0 =  mi_1.R_2_max
    mi_1.R_2_nominal_0 =  mi_1.R_2_nominal
    
    axes[0].set_ylim([-np.max(mi_1.T_u)*1.2,np.max(mi_1.T_u)*1.2])
    axes[1].set_ylim([-mi_1.P_n*1.5/1000,mi_1.P_n*1.5/1000])
    axes[0].set_xlim([0,mi_1.n_1*1.5])
    axes[1].set_xlim([0,mi_1.n_1*1.5])
    line_T_u_nom[0].set_data(n_array,(n_array-n_array) + mi_1.T_u_nom)
    line_I_1_nom[0].set_data(n_array,(n_array-n_array) + mi_1.I_nom)
    
    mi_1.R_2_start_0 =  mi_1.R_2_start
    mi_1.R_2_max_0 =  mi_1.R_2_max
    mi_1.R_2_nominal_0 =  mi_1.R_2_nominal



    sld_R2.value = 0
    sld_U1.value = mi_1.U_n
    sld_freq.value = mi_1.freq_nom
    ckbox_escalar.value = False
    
    update(change)
    
def update(change):
    if ckbox_escalar.value == True:
        sld_U1.value = sld_freq.value/50*400.0  
    mi_1.R_2_start =  mi_1.R_2_start_0 + sld_R2.value
    mi_1.R_2_max =  mi_1.R_2_max_0 +sld_R2.value
    mi_1.R_2_nominal =  mi_1.R_2_nominal_0+sld_R2.value
    mi_1.U_1 = sld_U1.value
    mi_1.freq = sld_freq.value
    n_array = np.hstack(([0],np.linspace(sld_freq.value*60/mi_1.N_pp*0.8,sld_freq.value*60/mi_1.N_pp*0.999,1000)))
    mi_1.update(n_array)

    idx_n = np.argmax(mi_1.T_u<sld_tau.value)
    n_interp = np.interp(n_array[idx_n], n_array, n_array)
    
    T_u_interp = np.interp(n_interp, n_array, mi_1.T_u)
    P_1_interp = np.interp(n_interp, n_array, np.abs(mi_1.P_1))
    Q_1_interp = np.interp(n_interp, n_array, np.abs(mi_1.Q_1))
    
    line_T_u_state[0].set_data(n_interp,mi_1.T_u[idx_n])
    #line_I_1_state[0].set_data(n_interp,I_1m_interp)
       
    c = np.abs(mi_1.I_1[idx_n])/mi_1.I_nom

    prog_c.bar_style = 'success'
    if c>1.1:
        prog_c.bar_style = 'warning'
    if c>1.2:
        prog_c.bar_style = 'danger'
    if c>0 and c<1.5:  
        prog_c.value = 100*c

    n_array = np.linspace(0,1499+200,200)
    mi_1.update(n_array)
    line_T_u[0].set_data(n_array, mi_1.T_u)
    line_P_1[0].set_data(n_array,mi_1.P_1/1000)
    line_Q_1[0].set_data(n_array,mi_1.Q_1/1000)
    fig.canvas.draw_idle()
    #fig.canvas.flush_events()

sld_R2.observe(update, names='value')
sld_U1.observe(update, names='value')
sld_freq.observe(update, names='value')
ckbox_escalar.observe(update, names='value')
drop_mtype.observe(initial, names='value')
btn_reset.on_click(initial)
sld_tau.observe(update, names='value')

layout_row1 = ipywidgets.HBox([fig.canvas])
layout_row2 = ipywidgets.HBox([drop_mtype,sld_R2,sld_U1])
layout_row3 = ipywidgets.HBox([sld_tau,ckbox_escalar,btn_reset,prog_c])
layout = ipywidgets.VBox([layout_row1,layout_row2,layout_row3])

initial(0)


layout

NameError: name 'line_I_1_nom' is not defined

In [8]:
mi_1.Q_1

array([169932.33220898, 169609.2775594 , 169286.35012941, 168963.54288134,
       168640.8485686 , 168318.25973235, 167995.76869796, 167673.36757145,
       167351.04823558, 167028.80234598, 166706.62132696, 166384.49636724,
       166062.41841547, 165740.37817555, 165418.3661018 , 165096.37239392,
       164774.38699173, 164452.39956972, 164130.39953137, 163808.37600327,
       163486.31782899, 163164.21356271, 162842.05146258, 162519.81948391,
       162197.50527195, 161875.09615452, 161552.57913429, 161229.9408808 ,
       160907.16772208, 160584.24563609, 160261.16024169, 159937.89678935,
       159614.44015147, 159290.7748123 , 158966.88485752, 158642.75396338,
       158318.36538543, 157993.70194677, 157668.74602594, 157343.4795442 ,
       157017.88395245, 156691.94021752, 156365.62880799, 156038.92967939,
       155711.82225891, 155384.28542936, 155056.29751266, 154727.83625245,
       154398.87879626, 154069.40167671, 153739.38079211, 153408.79138624,
       153077.60802722, 1

In [13]:
mi_1.P_n

22000.0